In [ ]:
#导包
import  pandas as pd
import numpy as np
import os
from fancyimpute import KNN
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
#朴素贝叶斯

from sklearn.naive_bayes import  BernoulliNB#伯努利

#k近邻
from sklearn.neighbors import KNeighborsClassifier

#随机森林填充缺失值
from sklearn.experimental import enable_iterative_imputer
#MICE多重差补
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

#miceforest实现多重插补
import miceforest as mf

#特征工程
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold
#欠采样与过采样
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
#随机调参
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.model_selection import cross_val_score#交叉验证
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report#分类报告
import xgboost as xgb

#多模融合bagging
from sklearn.ensemble import BaggingClassifier
#Stacking
from sklearn.ensemble import StackingClassifier
from sklearn import svm

#Visual Programming
import matplotlib.pyplot as plt
import seaborn as sns

#Feature Engineering
from sklearn.feature_selection import SelectKBest,chi2,f_classif

In [ ]:
df=pd.read_csv('C:\\Users\\zhang\\Desktop\\服务外包\\训练集\\train_10000.csv')
#读入数据
df=df.drop_duplicates()
filter_feature=[]
# 遍历每一列，过滤无用特征
for col in df.columns:
    # 检查该列是否所有值都为空
    if df[col].isnull().all():
        filter_feature.append(col)
        df.drop(col, axis=1)
    # 检查该列是否所有值都相同
    elif len(np.unique(df[col])) == 1:
        filter_feature.append(col)
        df.drop(col, axis=1) # 过滤无用的特性，value=0

filter_feature.append('sample_id')
df=df.drop(filter_feature,axis=1)

features = []
for x in df.columns: # 取特征
    if x not in filter_feature:
        features.append(x)
y=np.array(df["label"],dtype="int")

In [ ]:
train_df,test_df=train_test_split(df,test_size=0.3,random_state=42)
# 遍历每个特征并绘制KDE分布图
for col in train_df.columns:
    # 绘制训练集和测试集的KDE分布图
    sns.kdeplot(data=train_df[col], label='Train')
    sns.kdeplot(data=test_df[col], label='Test')

    # 添加图例和标题
    plt.legend()
    plt.title(f'KDE Plot of {col}')

    # 显示图形
    plt.show()

In [ ]:
#数据类型分类
int64_features=[]
float_features=[]
object_features=[]
category_features=[]
for col_name, col in df.iteritems():
    if col.dtype=='float64':
        float_features.append(col_name)
    elif col.dtype=='int64':
        int64_features.append(col_name)
    elif col.dtype=='category':
        category_features.append(col_name)
    else:
        object_features.append(col_name)

In [ ]:
#连续型和类别型
continuous_columns = df.select_dtypes(include=['float64']).columns
categorical_columns = df.select_dtypes(include=['int64']).columns

In [ ]:
#单变量分析，数据分布直方图
sns.set()

# 假设有一个名为df的数据框
n_features =105

# 设置画布大小
plt.figure(figsize=(10, n_features * 5))

for i, feature in enumerate(features):
    plt.subplot(n_features, 1, i + 1)
    sns.histplot(df[feature], kde=True)
    plt.title(feature)

plt.tight_layout()
plt.show()

In [ ]:
#双变量分析，相关系数的热力图
corr = df.corr()
plt.figure(figsize=(96, 72))
# 绘制热力图
sns.heatmap(corr,annot=True)
plt.show()

In [ ]:
#缺失值处理，使用miceforest库进行多重差补
# 创建内核数据集
kernel = mf.ImputationKernel(
  data=df,
  save_all_iterations=False,
  random_state=42
)

# 执行多重插补
kernel.mice(3)

# 获取插补后的数据
df= kernel.impute_new_data(new_data=df).complete_data(0)

In [ ]:
x=np.array(df.iloc[:,0:104],dtype="double")
# 创建一个随机森林回归器
estimator = RandomForestClassifier(random_state=42)

# 创建一个RFE实例
selector = RFE(estimator, n_features_to_select=23)

# 拟合数据
selector = selector.fit(x, y)

# 查看选定的特征
selected_features = selector.support_

# 更新数据集，只保留选定的特征
x = x[:, selected_features]

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [ ]:
#XGB随机调参
# 定义超参数分布
param_dist = {
    'n_estimators': randint(100, 1000),
    'learning_rate': uniform(0.01, 0.6),
    'max_depth': randint(2, 6),
    'min_child_weight': randint(1, 10),
    'subsample': uniform(0.5, 0.5),
    'gamma': uniform(0, 10),
    'colsample_bytree': uniform(0.5, 0.5),#表示均匀分布，最小值0.5，最大值1
    'reg_lambda': uniform(0.1, 10)
}

# 创建一个XGBClassifier实例
model = xgb.XGBClassifier(objective='multi:softmax', num_class=6)

# 创建一个RandomizedSearchCV实例
search = RandomizedSearchCV(model, param_distributions=param_dist,
                            n_iter=25, scoring='f1', cv=5)

# 拟合数据
search.fit(x, y)

# 查看最优超参数组合
best_params = search.best_params_
best_params

In [ ]:
base_estimators = [('svc', SVC()),
                   ('dt', DecisionTreeClassifier()),
                   ('mlp',MLPClassifier( solver='adam', learning_rate= 'adaptive', hidden_layer_sizes= (128, 64), activation= 'tanh')),
                   ('xgb',xgb.XGBClassifier(objective='multi:softmax', num_class=6,**best_params))
]

# 创建元分类器
meta_estimator = LogisticRegression()

# 创建Stacking分类器
stacking = StackingClassifier(estimators=base_estimators, final_estimator=meta_estimator)

# 拟合数据
stacking.fit(x_train, y_train)

# 预测数据
y_pred = stacking.predict(x_test)

report = classification_report(y_test, y_pred, output_dict=True)# 提取分类报告中的精确率、召回率和 F1 分数
macro_precision = report['macro avg']['precision']
macro_recall = report['macro avg']['recall']
# 计算 MacroF1
macro_f1 = 2 * (macro_precision * macro_recall) / (macro_precision + macro_recall)
# 打印MacroF1
print("MacroF1:", macro_f1)

In [ ]:
#验证数据集
df_test=pd.read_csv('C:\\Users\\zhang\\Desktop\\服务外包\\验证集\\validate_1000.csv')
df=pd.read_csv('C:\\Users\\zhang\\Desktop\\服务外包\\训练集\\train_10000.csv')
y_test_test=np.array(df_test["label"],dtype="int")
df_test=df_test.drop(filter_feature,axis=1)
df=df.drop(filter_feature,axis=1)

In [ ]:
# 创建内核数据集
kernel = mf.ImputationKernel(
  data=df_test,
  save_all_iterations=False,
  random_state=42
)

# 执行多重插补
kernel.mice(3)

# 获取插补后的数据
df_test= kernel.impute_new_data(new_data=df_test).complete_data(0)

In [ ]:
x_test_True=np.array(df_test.iloc[:,0:104],dtype="double")
x_test_True=x_test_True[:,selected_features]

In [ ]:
import json

df_test_True= pd.read_json('C:\\Users\\zhang\\Desktop\\服务外包\\验证集\\validate_1000_y_true.json', typ='series')
df_test_True= df_test_True.to_frame().reset_index()
df_test_True.columns = ['Sample_id', 'y_label']
df_test_True
y_True=np.array(df_test_True["y_label"],dtype="int")#读入实际的y_label

y_pred_True = stacking.predict(x_test_True)
report = classification_report(y_True, y_pred_True, output_dict=True)# 提取分类报告中的精确率、召回率和 F1 分数
macro_precision = report['macro avg']['precision']
macro_recall = report['macro avg']['recall']
# 计算 MacroF1
macro_f1 = 2 * (macro_precision * macro_recall) / (macro_precision + macro_recall)
# 打印MacroF1
print("MacroF1:", macro_f1)

In [ ]:
scores = cross_val_score(stacking, x_test_True, y_True, cv=10) # 10折交叉验证
print(scores)